<a href="https://colab.research.google.com/github/monk1108/deep_learning/blob/master/5_ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ---------------------------------------------------------------------------- #
# An implementation of https://arxiv.org/pdf/1512.03385.pdf                    #
# See section 4.2 for the model architecture on CIFAR-10                       #
# Some part of the code was referenced from below                              #
# https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py   #
# ---------------------------------------------------------------------------- #

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
num_epochs = 80
batch_size = 100
learning_rate = 0.001

# Image preprocessing modules
#详见官方文档https://pytorch.org/docs/stable/torchvision/transforms.html?highlight=transforms%20compose#torchvision.transforms.Compose
transform = transforms.Compose([
  transforms.Pad(4),  #Pad the given image on all sides with the given “pad” value. 
  transforms.RandomHorizontalFlip(),  #Horizontally flip the given image randomly with a given probability.
  transforms.RandomCrop(32),  #Crop the given image at a random location.    size: Desired output size of the crop
  transforms.ToTensor()
])

# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=False, 
                                            transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

Extracting ../../data/cifar-10-python.tar.gz to ../../data/


In [7]:
# 3x3 convolution
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)

# Residual block
class ResidualBlock(nn.Module):   #单个残差网络块
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample    #知乎：你看那个基础结构里不是有一个从x直接连到下面的线吗，
    #如果上一个ResidualBlock的输出维度和当前的ResidualBlock的维度不一样，那就对这个x进行downSample操作，
    #如果维度一样，直接加就行了，这时直接out+=residual
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

In [8]:
# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
        
    #identity mapping这个直连的维度和F(x) 输出的维度不一样无法直接相加的问题：
    #采用一个kernel=1的conv2d卷积核融合并降低通道信息，如果H/W尺度也不一样就设计stride。
    def make_layer(self, block, out_channels, blocks, stride=1):
      #这里的block就是上文定义的ResidualBlock
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [9]:
model = ResNet(ResidualBlock, [2, 2, 2]).to(device)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# For updating learning rate
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

# Train the model
total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'resnet.ckpt')

Epoch [1/80], Step [100/500] Loss: 1.5073
Epoch [1/80], Step [200/500] Loss: 1.5220
Epoch [1/80], Step [300/500] Loss: 1.3331
Epoch [1/80], Step [400/500] Loss: 1.3245
Epoch [1/80], Step [500/500] Loss: 1.2732
Epoch [2/80], Step [100/500] Loss: 1.0770
Epoch [2/80], Step [200/500] Loss: 1.0567
Epoch [2/80], Step [300/500] Loss: 1.0133
Epoch [2/80], Step [400/500] Loss: 1.0819
Epoch [2/80], Step [500/500] Loss: 1.0909
Epoch [3/80], Step [100/500] Loss: 0.9125
Epoch [3/80], Step [200/500] Loss: 0.8477
Epoch [3/80], Step [300/500] Loss: 0.9606
Epoch [3/80], Step [400/500] Loss: 0.6902
Epoch [3/80], Step [500/500] Loss: 0.6802
Epoch [4/80], Step [100/500] Loss: 0.7447
Epoch [4/80], Step [200/500] Loss: 0.6421
Epoch [4/80], Step [300/500] Loss: 0.7123
Epoch [4/80], Step [400/500] Loss: 0.7003
Epoch [4/80], Step [500/500] Loss: 0.8345
Epoch [5/80], Step [100/500] Loss: 0.5385
Epoch [5/80], Step [200/500] Loss: 0.7442
Epoch [5/80], Step [300/500] Loss: 0.7469
Epoch [5/80], Step [400/500] Loss: